In [2]:
import torch
import random 
import numpy as np 
from tqdm import tqdm 
from scipy.spatial.distance import cdist, cosine
from scipy.optimize import linear_sum_assignment
from utils.corrupt_graph import remove_edge, remove_node, add_edge, add_node
from utils.query_machine import get_candidates
from python_emb import *
import torch.nn.functional as F
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, spearmanr
from collections import defaultdict

In [6]:
for dataset in ['yeast']:
    for dim in [4, 8, 16, 32, 64, 128, 512]:
        ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

        ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
        ori_emb_model = ori_emb_model.cuda()
        ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_50_dim{}.pt'.format(dataset, dim)))
        ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
        ori_emb_model.eval()

        ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
        ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


        for i, node in enumerate(ori_graph_data.G.nodes):
            ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

        query_machine = GraphQuery(ori_emb_model, 
                ori_graph_emb,
                ori_graph_data.G,
                ori_graph_data.id_map,
                ori_graph_data.feats,
                ori_graph_data.raw_feats,
                ori_graph_data.full_adj,
                ori_graph_data.deg)

        def get_embedding_subgraph(subgraph):
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
            embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

            return embedding_subgraph, sub_id_map


        mcs2emb = defaultdict(list)
        GRAPH_SIZE = 20
        N_REMOVE = 3
        N_CORE = 1000

        if N_CORE<= len(query_machine.ori_graph):
            core_nodes = random.sample([node for node in query_machine.ori_graph], N_CORE)
        else:
            core_nodes = random.choices([node for node in query_machine.ori_graph], k=N_CORE)
        for i, core_node in tqdm(enumerate(core_nodes)):
            biggraph = query_machine.create_subgraph_from_core(core_node, GRAPH_SIZE)
            if len(biggraph) != GRAPH_SIZE:
                continue
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

            embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_biggraph = embedding_biggraph.detach().cpu().numpy()
            for dist in range(1,8):
                connected = False
                n_tries  = 0
                while(not connected and n_tries<=10):
                    subgraph = remove_node(biggraph, dist)
                    n_tries += 1
                    if len(subgraph)==0: continue
                    connected = nx.is_connected(subgraph)
                    if (n_tries>10): break
                if n_tries <=10 and nx.is_connected(subgraph):
                    sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
                    embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
                    embedding_subgraph = embedding_subgraph.detach().cpu().numpy()
                    emb_sim = ((embedding_biggraph.mean(0)- embedding_subgraph.mean(0))**2).sum()
                    if not np.isnan(emb_sim):
                        mcs2emb[dist].append(emb_sim)
        X = []
        Y = []
        for k,v in mcs2emb.items():
            X.append(k)
            Y.append(sum(v)/len(v))
        print(dataset, dim, spearmanr(X, Y), pearsonr(X,Y))


Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


1000it [00:23, 41.80it/s]


yeast 4 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9936700555275177, 6.102099533530004e-06)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


1000it [00:24, 40.39it/s]


yeast 8 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9920663818006499, 1.0722196458243565e-05)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


...

yeast 64 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.990342519155756, 1.7513094062956154e-05)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


1000it [00:25, 39.98it/s]


yeast 128 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9938755529832741, 5.619460897373712e-06)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742


1000it [00:24, 40.18it/s]

yeast 512 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9942324435657697, 4.837158105324026e-06)


In [7]:
for dataset in ['human', 'cora', 'citeseer', 'pubmed','wordnet']:
    for dim in [4, 8, 16, 32, 64, 128, 512]:
        ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

        ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
        ori_emb_model = ori_emb_model.cuda()
        ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_50_dim{}.pt'.format(dataset, dim)))
        ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
        ori_emb_model.eval()

        ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
        ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


        for i, node in enumerate(ori_graph_data.G.nodes):
            ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

        query_machine = GraphQuery(ori_emb_model, 
                ori_graph_emb,
                ori_graph_data.G,
                ori_graph_data.id_map,
                ori_graph_data.feats,
                ori_graph_data.raw_feats,
                ori_graph_data.full_adj,
                ori_graph_data.deg)

        def get_embedding_subgraph(subgraph):
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
            embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

            return embedding_subgraph, sub_id_map


        mcs2emb = defaultdict(list)
        GRAPH_SIZE = 20
        N_REMOVE = 3
        N_CORE = 1000

        if N_CORE<= len(query_machine.ori_graph):
            core_nodes = random.sample([node for node in query_machine.ori_graph], N_CORE)
        else:
            core_nodes = random.choices([node for node in query_machine.ori_graph], k=N_CORE)
        for i, core_node in tqdm(enumerate(core_nodes)):
            biggraph = query_machine.create_subgraph_from_core(core_node, GRAPH_SIZE)
            if len(biggraph) != GRAPH_SIZE:
                continue
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

            embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_biggraph = embedding_biggraph.detach().cpu().numpy()
            for dist in range(1,8):
                connected = False
                n_tries  = 0
                while(not connected and n_tries<=10):
                    subgraph = remove_node(biggraph, dist)
                    n_tries += 1
                    if len(subgraph)==0: continue
                    connected = nx.is_connected(subgraph)
                    if (n_tries>10): break
                if n_tries <=10 and nx.is_connected(subgraph):
                    sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
                    embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
                    embedding_subgraph = embedding_subgraph.detach().cpu().numpy()
                    emb_sim = ((embedding_biggraph.mean(0)- embedding_subgraph.mean(0))**2).sum()
                    if not np.isnan(emb_sim):
                        mcs2emb[dist].append(emb_sim)
        X = []
        Y = []
        for k,v in mcs2emb.items():
            X.append(k)
            Y.append(sum(v)/len(v))
        print(dataset, dim, spearmanr(X, Y), pearsonr(X,Y))


Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/human/human-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/human/human-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/human/human-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 86282
Preprocessing finished, graph info:
Name: human
Type: Graph
Number of nodes: 4674
Number of edges: 86282
Average degree:  36.9200


1000it [00:24, 41.24it/s]


human 4 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9919182506599817, 1.122882728796473e-05)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/human/human-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/human/human-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/human/human-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 86282
Preprocessing finished, graph info:
Name: human
Type: Graph
Number of nodes: 4674
Number of edges: 86282
Average degree:  36.9200


1000it [00:23, 42.35it/s]


human 8 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9927816469569724, 8.4696928369132e-06)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/human/human-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/human/human-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/human/human-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 86282
Preprocessing finished, graph info:
Name: human
Type: Graph
Number of nodes: 4674
Number of edges: 86282
Average degree:  36.9200


...

wordnet 64 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9857879136988221, 4.589695088312876e-05)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


1000it [00:24, 40.58it/s]


wordnet 128 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.996208723310261, 1.6964259246497103e-06)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755


1000it [00:24, 41.57it/s]

wordnet 512 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9941223802588799, 5.070942741697384e-06)
